In [1]:
import os
import sys
import pandas
import numpy

import findspark
findspark.init("/usr/local/spark/spark")

import pyspark
from pyspark.sql.window import Window
import pyspark.sql.functions as func

from pyspark.rdd import reduce
from pyspark.sql.types import DoubleType
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import GaussianMixture, KMeansModel, KMeans
from pyspark.ml.linalg import SparseVector, VectorUDT, Vector, Vectors

In [2]:
from pyspark.sql.functions import udf, col, struct
from pyspark.sql.types import ArrayType, DoubleType, StringType
from pyspark.mllib.linalg.distributed import RowMatrix, DenseMatrix

In [68]:
cluster_files =  "/Users/simondi/PROJECTS/target_infect_x_project/src/tix-analysis/data/outlier-removal"

In [3]:
conf = pyspark.SparkConf().setMaster("local[*]").set("spark.driver.memory", "4G").set("spark.executor.memory", "4G")
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.sql.SparkSession(sc)

In [4]:
spark.version

'2.3.0'

In [8]:
from scipy.stats import multivariate_normal

In [28]:
k = 15
svd = multivariate_normal.rvs(mean=scipy.ones(k), size=1000)

In [69]:
dff = map(lambda x: (int(x[0]), Vectors.dense(x[1:])), svd)
df = spark.createDataFrame(dff, schema=["label", "features"])

In [70]:
df = spark.read.parquet(cluster_files)

In [73]:
s = df.select("features")

In [138]:
def to_array(col):
    def to_array_(v):
        return v.toArray().tolist()

    return udf(to_array_, ArrayType(DoubleType()))(col)

len_vec = len(s.select("features").take(1)[0][0])
data = (s.withColumn("f", to_array(col("features")))
        .select([col("f")[i] for i in range(5)]))

In [139]:
assembler = VectorAssembler(inputCols=data.columns, outputCol='features')

In [140]:
data = assembler.transform(data)

In [147]:
datas = data.limit(70)

In [148]:
for i in range(2, 10 + 1):
    time1 = time.time()
    km = GaussianMixture(k=i, seed=23)
    model = km.fit(datas)
    time2 = time.time()
    print(i, model.summary.logLikelihood, time2 - time1)

2 -326.46525564568196 0.8249571323394775
3 -311.5342285986531 0.5016491413116455
4 -294.052483271471 0.5052499771118164
5 -263.9393097144451 0.47350287437438965
6 -206.20980517884834 0.36144089698791504
7 -216.66304478449942 0.6501719951629639
8 -103.86915656626311 0.37635302543640137
9 8.433293501608464 0.47348713874816895
10 234.48800703311113 0.624981164932251


In [156]:
model.gaussiansDF.take(2)[1][1]

DenseMatrix(4, 4, [0.2462, 0.0275, -0.193, -0.2179, 0.0275, 0.2935, -0.366, 0.0703, -0.193, -0.366, 0.8844, 0.0509, -0.2179, 0.0703, 0.0509, 0.2532], False)

In [162]:
svd = multivariate_normal.rvs(mean=scipy.ones(5), size=100)

In [172]:
df.take(1)

[Row(label=0, features=DenseVector([0.2978, 1.4635, 0.4609, -0.5065, 1.8492]))]

In [179]:
scipy.random.seed(100)
sc = [multivariate_normal.rvs(mean=scipy.ones(i), size=10) for i in range(10)]
dff = map(lambda x: (int(x[0]), Vectors.dense(x[0:])), sc)
df = spark.createDataFrame(dff, schema=["label", "features"])
for i in [100, 90, 80, 70, 60]:
    time1 = time.time()
    km = GaussianMixture(seed=5, k=10)
    model = km.fit(df.limit(i))
    time2 = time.time()
    print(i,model.summary.logLikelihood, time2 - time1)

100 -443.5606872205738 0.23123884201049805
90 -292.6965047310385 0.3216116428375244
80 -212.75029002979562 0.3197188377380371
70 -44.16679162064955 0.2414848804473877
60 219.89197421451937 0.22657322883605957


In [151]:

df.take(1)

[Row(label=0, features=DenseVector([1.2011, 0.7218, 0.0537, 2.2096]))]

In [41]:
km = GaussianMixture().setK(5).setSeed(23)
model = km.fit(df)

In [56]:
model.gaussiansDF

DataFrame[mean: vector, cov: matrix]

In [58]:
model.weights

[0.14620840581414205,
 0.15119357104416584,
 0.08541240437382741,
 0.255908435981313,
 0.3612771827865516]

In [ ]:
s